# Optuna와 파이프라인을 활용한 하이퍼파라미터 최적화 노트북

이 Jupyter Notebook은 `scikit-learn`의 유방암 데이터셋(`load_breast_cancer`)을 사용하여 **Optuna** 라이브러리를 통해 `RandomForestClassifier` 모델의 최적 하이퍼파라미터를 탐색하는 과정을 보여줍니다. 특히, 데이터 전처리(스케일링)와 모델 학습을 **파이프라인(`Pipeline`)**으로 통합하여 더욱 체계적인 워크플로우를 구성합니다.

**Optuna 개요:**
* **자동화된 하이퍼파라미터 최적화**: 모델의 성능을 최대화하는 최적의 하이퍼파라미터 조합을 효율적으로 찾아줍니다.
* **유연성**: 다양한 머신러닝 모델과 사용자 정의 목적 함수에 적용할 수 있습니다.
* **시각화**: 최적화 과정을 시각적으로 분석할 수 있는 도구를 제공합니다.
* **병렬 처리**: 여러 스터디를 병렬로 실행하여 최적화 시간을 단축할 수 있습니다.

**파이프라인(`Pipeline`) 개요:**
* **전처리 및 모델 통합**: 여러 전처리 단계와 최종 모델을 하나의 객체로 묶어줍니다.
* **코드 간소화**: 반복적인 전처리 코드를 줄이고 워크플로우를 명확하게 만듭니다.
* **데이터 누수 방지**: 훈련 데이터에만 전처리를 적용하고 테스트 데이터에는 훈련 데이터의 통계량으로 변환하는 등 데이터 누수를 효과적으로 방지합니다.

---
## 1. 라이브러리 임포트

필요한 라이브러리들을 임포트합니다.

In [ ]:
import optuna
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris, load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC # 하이퍼파라미터 개수가 많아서 선택함
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline # 파이프라인 추가
from sklearn.preprocessing import StandardScaler, MinMaxScaler # 스케일러 추가
# classification_report: 분류 중에서도 이진 분류 평가 라이브러리
# accuracy_score: 단순히 정확도 판단 기준
# GridSearchCV: 파라미터를 주면 각 파라미터별로 전체 조합을 만들어서 다 돌려본다.

---
## 2. 데이터 로드 및 전처리

`load_breast_cancer` 데이터셋을 로드하고, 타겟 레이블을 반전하여 0을 양성(benign), 1을 악성(malignant)으로 설정합니다. 데이터의 특성 개수, 샘플 개수, 클래스 분포를 확인합니다.

In [ ]:
cancer = load_breast_cancer() # iris 대신 cancer로 변수명 변경 (원본 코드와 일치)
# cancer.data, cancer.target => 데이터프레임으로
X = pd.DataFrame(cancer.data, columns=cancer.feature_names)
y = cancer.target # 0이 악성, 1이 양성 => 둘의 값을 반전하자. 나중에 모델 평가 해석시에 그게 더 편함
y_change = np.where(y==0, 1, 0) # 0->1 로 1->0으로 바꿈

print("유방암 데이터셋 정보:")
print(f"특성 개수 {X.shape[1]}")
print(f"샘플 개수 {X.shape[0]}")
print(f"클래스 분포 (0:양성 1:악성) {dict(zip(*np.unique(y_change, return_counts=True) ))}")

---
## 3. 데이터 분할

데이터 불균형을 고려하여 `stratify` 옵션을 사용하여 훈련 세트와 테스트 세트를 분할합니다. 이는 각 클래스의 비율이 훈련 세트와 테스트 세트에서 동일하게 유지되도록 합니다.

In [ ]:
# 악성인 사람과 양성인 사람 간의 데이터 불균형
# 불균형 데이터셋일 경우에 훈련셋과 테스트셋을 쪼갤 때 그 균형을 유지하면서 쪼개라
# stratify=y_change
X_train, X_test, y_train, y_test = train_test_split(X, y_change, random_state=1234,
                                                    test_size=0.2, stratify=y_change)
print("훈련 데이터셋:")
print(X_train.shape, X_test.shape)
print(f"y_train 분포: {dict(zip(*np.unique(y_train, return_counts=True) ))}")
print(f"y_test 분포: {dict(zip(*np.unique(y_test, return_counts=True) ))}")

---
## 4. Optuna 목적 함수 정의 (파이프라인 포함)

Optuna가 최적화할 목적 함수(`objective`)를 정의합니다. 이 함수는 Optuna가 탐색할 하이퍼파라미터의 범위를 설정하고, 해당 하이퍼파라미터로 `RandomForestClassifier` 모델을 포함하는 **파이프라인**을 학습시킨 후 테스트 세트에서의 정확도를 반환합니다.

여기서는 `StandardScaler`를 사용하여 데이터를 스케일링하는 단계를 파이프라인에 추가했습니다.

In [ ]:
def objective(trial): # 변수명은 마음대로
    # 옵투나를 통해 탐색할 하이퍼파라미터 범위를 정의한다
    
    max_depth = trial.suggest_int('classifier__max_depth', 5, 20) # 그리드서치 5,6,7,8,9,...20 => 시작, 엔딩
    # 트리의 최대 깊이
    min_samples_leaf = trial.suggest_int('classifier__min_samples_leaf', 1, 10)
    # 리프 노드가 되기 위한 최소 샘플 수
    min_samples_split = trial.suggest_int('classifier__min_samples_split', 2, 10)
    n_estimators = trial.suggest_int('classifier__n_estimators', 50, 100)

    # 스케일러 선택 (StandardScaler 또는 MinMaxScaler)
    scaler_name = trial.suggest_categorical('scaler', ['StandardScaler', 'MinMaxScaler'])
    scaler = StandardScaler() if scaler_name == 'StandardScaler' else MinMaxScaler()

    model = RandomForestClassifier(max_depth=max_depth,
                                 min_samples_leaf=min_samples_leaf,
                                 min_samples_split=min_samples_split,
                                 n_estimators=n_estimators,
                                 random_state=42,
                                 n_jobs=-1) # 내부 프로세스 CPU 개수 *2라서 -1을 주면 알아서 최대치를 사용한다
    
    # 파이프라인 정의
    pipeline = Pipeline(
        [('scaler', scaler),
         ('classifier', model)]
    )
    
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred) # 예측 정확도
    return accuracy # 반드시 마지막에 리턴해야 한다. 목적값

---
## 5. Optuna 스터디 실행

Optuna 스터디를 생성하고 `objective` 함수를 호출하여 하이퍼파라미터 최적화를 수행합니다. `direction="maximize"`는 정확도를 최대화하는 방향으로 최적화를 진행함을 의미합니다. `n_trials`는 시도할 횟수를 지정합니다.

In [ ]:
# 옵투나 스터디 생성
study = optuna.create_study(direction="maximize") # 이익을 최대화하는 방향으로 study 객체를 만든다
print("옵투나 최적화 시작 (50회 시도)")
study.optimize(objective, n_trials=50) # 콜백 함수, 횟수를 지정한다
# optimize - 최적화 함수

print(f"최고 정확도: {study.best_trial.value}")
print(f"최적 하이퍼파라미터: {study.best_trial.params}")

---
## 6. 추가 학습: 캘리포니아 주택 가격 데이터셋

회귀 문제에 대한 파이프라인과 `GridSearchCV` 복습을 위해 `fetch_california_housing()` 데이터셋을 활용해 볼 수 있습니다. 이 데이터셋은 캘리포니아 주택 가격 예측에 사용되며, 다양한 회귀 모델과 전처리 기법을 적용하기 좋습니다.

```python
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# 데이터 로드
housing = fetch_california_housing()
X_housing = housing.data
y_housing = housing.target

# 훈련/테스트 세트 분할
X_train_h, X_test_h, y_train_h, y_test_h = train_test_split(X_housing, y_housing, random_state=42)

# 파이프라인 정의
pipeline_reg = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])

# GridSearchCV를 위한 파라미터 그리드
param_grid_reg = {
    'scaler': [StandardScaler(), MinMaxScaler()], # 다른 스케일러 시도
    'regressor__fit_intercept': [True, False] # 선형 회귀의 fit_intercept 파라미터
}

# GridSearchCV 실행
grid_search_reg = GridSearchCV(pipeline_reg, param_grid_reg, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_reg.fit(X_train_h, y_train_h)

print("최적 파라미터 (캘리포니아 주택):", grid_search_reg.best_params_)
print("최고 교차 검증 점수 (캘리포니아 주택):", -grid_search_reg.best_score_)
print("테스트 세트 점수 (캘리포니아 주택):", grid_search_reg.score(X_test_h, y_test_h))
```